<a href="https://colab.research.google.com/github/mabonmn/Surgical_Video_Analysis/blob/main/Informative_and_non-informative_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages



In [ ]:
#MADE BY MABON MANOJ NINAN - UNIVERSITY OF CINCINATI
#MEDHI NOURIZI

import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
import os
import re
import time
import shutil
from sklearn.cluster import KMean

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
vid_path='/content/gdrive/MyDrive/Feature Extraction - Mabon/Surgical Video_p7/LDN.mpg'

Mounted at /content/gdrive


#Paths And Temp Folder

In [ ]:
path='/content/temp'
out='/content/output_dir'
isExists = os.path.exists(path)
isExists2 = os.path.exists(out)
if not isExists:
  os.makedirs(path)
  print(" The directory was created successfully")
else:
  print(" Directory already exists")
if not isExists2:
  os.makedirs(out)
  print(" The directory was created successfully")
else:
  print(" Directory already exists")

 Directory already exists
 The directory was created successfully


#Segments from video that contain the actual surgery

In [ ]:
start_time = [0, 222, 514, 1008, 1328, 1636, 2132, 2464, 2976, 5589, 5630, 5668]
end_time = [160, 425, 629, 1176, 1444, 1741, 2180, 2513, 3013, 5615, 5650, 5685]

#Extracting the frames form the video According to the Predifined Useful segments


In [ ]:
cap = cv2.VideoCapture(vid_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
total_seconds = total_frames / fps


if isinstance(start_time, list) and isinstance(end_time, list) and len(start_time) != len(end_time):
    raise ValueError("Error! length of start time list must be equal to length of end time list")

if isinstance(start_time, int):
    start_time = [start_time]
if isinstance(end_time, int):
    end_time = [end_time]
if not start_time:
    start_time = [0]
if not end_time:
    end_time = [total_seconds]

check_list = [start <= end for start, end in zip(start_time, end_time)]
if max(end_time) > total_seconds:
    raise ValueError("Error! time in end time list exceeds total seconds")
elif not min(check_list):
    raise ValueError("Error! start time is bigger than end time")

# Initialization
frameCount = 0

start_frame_list = [round(time * fps) for time in start_time]
end_frame_list = [round(time * fps) for time in end_time]
duration_of_frame = round(fps / 5)

print("Original FPS: ", fps)
print("Extracted FPS: ", 5)
print("Number of frames: ", total_frames)
print("Total seconds: ", total_seconds)
print("Extracting...")


Original FPS:  29.97002997002997
Extracted FPS:  5
Number of frames:  238780.0
Total seconds:  7967.292666666667
Extracting...
Finish


In [ ]:
# Extract frames from file
for start, end in zip(start_frame_list, end_frame_list):  # what does zip do?
    cap.set(cv2.CAP_PROP_POS_FRAMES, start - 1)
    while start < end:
        ret, image = cap.read()
        if not ret:
            break
        cv2.imwrite(path + '/frame' + str(frameCount) + '.jpg', image)
        start += duration_of_frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start - 1)
        frameCount += 1
            
        if frameCount % 100 == 0:
          print('Finish', frameCount, 'frames')


#GLCM Gray-Level Co-Occurrence Matrix

In [ ]:
#GLCM Gray-Level Co-Occurrence Matrix
def maxGrayLevel(img):
    max_gray_level=0
    (height,width)=img.shape

    for y in range(height):
        for x in range(width):
            if img[y][x] > max_gray_level:
                max_gray_level = img[y][x]
                
    return max_gray_level+1


In [ ]:
def getGlcm(input,d_x,d_y):
    srcdata=input.copy()
    ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
    (height,width) = input.shape

    max_gray_level=maxGrayLevel(input)

    #if the number of gray levels is greater than gray_level, reduce the gray level of the image to gray_level and reduce the size of the gray level co-occurrence matrix

    if max_gray_level > gray_level:
        for j in range(height):
            for i in range(width):
                srcdata[j][i] = srcdata[j][i]*gray_level / max_gray_level

    for j in range(height-d_y):
        for i in range(width-d_x):
             rows = srcdata[j][i]
             cols = srcdata[j + d_y][i+d_x]
             ret[rows][cols]+=1

    for i in range(gray_level):
        for j in range(gray_level):
            ret[i][j]/=float(height*width)

    return ret


#Feature Computation

In [ ]:

def feature_computer(p, feature):
    Con=0.0                                                           # Contrast
    Ent=0.0                                                           # Entropy
    Asm=0.0                                                           # Angular Second Moment 
    Idm=0.0                                                           # Inverse Differential Moment
    Eng=0.0                                                           # Energy
    Corr=0.0                                                          # Correlation
    Dis=0.0                                                           # Dissimilarity
    Uni=0.0                                                           # Uniformity
    for i in range(gray_level):
        for j in range(gray_level):
            Con += (i-j)*(i-j)*p[i][j]
            Asm += p[i][j]*p[i][j]
            Idm += p[i][j]/(1+(i-j)*(i-j))
            Dis += p[i][j]*abs(i-j)
            
            if (1-abs(i-j)) > 0:
                Uni += p[i][j]/(1-abs(i-j))
        
            if p[i][j]>0.0:
                Ent+=p[i][j]*math.log(p[i][j])
                
    Eng = Asm**0.5
    
            
    if feature == 'contrast':
        return Con
    elif feature == 'entropy':
        return -Ent
    elif feature == 'ASM':
        return Asm
    elif feature == 'IDM':
        return Idm
    elif feature == 'energy':
        return Eng
    elif feature == 'dissimilarity':
        return Dis
    elif feature == 'uniformity':
        return Uni
    else:
        print('\033[41m' + "Error! Feature must be contrast, entropy, ASM, IDM, energy, dissimilarity or uniformity" + '\033[0m')

#Main Funtion

In [ ]:
input_dir = path
img_paths = []
for i in range(int(frameCount)):
    if os.path.isfile(input_dir + '/frame' + str(i) +'.jpg'):
        img_paths.append(input_dir + r'/frame' + str(i) +'.jpg')
    else:
        print('frame' + str(i) +'.jpg'+"doesn't in the directory")

In [ ]:
startTime = time.time()

feature_list = []
counter = 0

#Define the maximum number of gray levels

gray_level = 16 
for img_path in img_paths:
    counter += 1
    img = cv2.imread(img_path,0)
    
    # get 2D fft magnitude spectrum
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    cv2.imwrite(out+'/magnitude_spectrum.bmp', magnitude_spectrum) # output the 2D-fft spectrum image

    
    # get GLCM and features
    img_spectrum = cv2.imread(out+'/magnitude_spectrum.bmp', 0) # read the 2D-fft spectrum image
    img_shape = img_spectrum.shape
    img_spectrum = cv2.resize(img,(int(img_shape[1]/2),int(img_shape[0]/2)),interpolation=cv2.INTER_CUBIC)
    
    # Different GLCM direction
    glcm_0 = getGlcm(img_spectrum, 1,0)
    glcm_1 = getGlcm(img_spectrum, 0,1)
    glcm_2 = getGlcm(img_spectrum, 1,1)

    # GLCM features
    con = feature_computer(glcm_0, 'contrast')
    ent = feature_computer(glcm_0, 'entropy')
    ASM = feature_computer(glcm_0, 'ASM')
    #IDM = feature_computer(glcm_0, 'IDM')
    eng = feature_computer(glcm_0, 'energy')
    dis = feature_computer(glcm_0, 'dissimilarity')
    #uni = feature_computer(glcm_0, 'uniformity')
    
    feature_list.append([con, ent, ASM, eng, dis]) 
    
    if counter % 500 == 0:
        print('Finish', counter, 'frames')


timeCost = time.time()-startTime
print('Fetch the feature list costs : %d minutes %d seconds' %((timeCost // 60), (timeCost % 60)))

Finish 100 frames
Finish 200 frames
Finish 300 frames
Finish 400 frames
Finish 500 frames
Finish 600 frames
Finish 700 frames
Finish 800 frames
Finish 900 frames
Finish 1000 frames
Finish 1100 frames
Finish 1200 frames
Finish 1300 frames
Finish 1400 frames
Finish 1500 frames
Finish 1600 frames
Finish 1700 frames
Finish 1800 frames
Finish 1900 frames
Finish 2000 frames
Finish 2100 frames
Finish 2200 frames
Finish 2300 frames
Finish 2400 frames
Finish 2500 frames
Finish 2600 frames
Finish 2700 frames
Finish 2800 frames
Finish 2900 frames
Finish 3000 frames
Finish 3100 frames
Finish 3200 frames
Finish 3300 frames
Finish 3400 frames
Finish 3500 frames
Finish 3600 frames
Finish 3700 frames
Finish 3800 frames
Finish 3900 frames
Finish 4000 frames
Finish 4100 frames
Finish 4200 frames
Finish 4300 frames
Finish 4400 frames
Finish 4500 frames
Finish 4600 frames
Finish 4700 frames
Finish 4800 frames
Finish 4900 frames
Finish 5000 frames
Finish 5100 frames
Finish 5200 frames
Finish 5300 frames
Fe

#KMeans Clustering of Features

In [ ]:
startTime = time.time()

k = 3
kmeans = KMeans(n_clusters=k)
kmeans.fit(feature_list)
label = kmeans.labels_

timeCost = time.time() - startTime
print('Training this model costs : %d minutes %d seconds' %((timeCost // 60), (timeCost % 60)))

Training this model costs : 0 minutes 0 seconds


#Folders For Informative and Non-informative features

In [ ]:
#Delete all folder then create new one
for x in range(k):
    if os.path.isdir(out+r'/a'+str(x)):
        shutil.rmtree(out+r'/a'+str(x))
        os.mkdir(out+r'/a'+str(x))
    else:
        os.mkdir(out+r'/a'+str(x))
      

#Sorting of Frames

In [ ]:
#%% save the result of cluster
startTime = time.time()
i=0
cluster_result = []
for path1 in img_paths:
    image = cv2.imread(path1)   
    for x in range(k):
        if label[i] == x:
          cv2.imwrite(out +'/a'+str(x)+'/frane' +str(i) + '.jpg', image)
          break
        else:
            continue
    i=i+1
timeCost = time.time()-startTime
print('Save the classified results costs : %d minutes %d seconds' %((timeCost // 60), (timeCost % 60)))

Save the classified results costs : 0 minutes 17 seconds


#Download frames

In [ ]:
from google.colab import files
dir_to_zip = '/content/output_dir/'
output_filename = 'file1.zip' 
delete_dir_after_download = "No" 
os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )
if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )
files.download( output_filename )


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>